# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
school_data.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [3]:
student_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [4]:
#extract the individual values
tot_schools = school_data.shape[0]
print("Schools: " + str(tot_schools))
tot_students = student_data.shape[0]
print("Students:" + str(tot_students))
tot_budget = school_data["budget"].sum()
print("Budget: " + str(tot_budget))
math_average = student_data["math_score"].mean()
print(f"Average math: {math_average}")
reading_average = student_data["reading_score"].mean()
print(f"Average reading: {reading_average}")
overall_average = (math_average + reading_average)/2
print(f"Overall Average: {overall_average}")
percent_pass_math = student_data.loc[student_data["math_score"]>=70].shape[0]/tot_students
print(f"% Passing Math: {percent_pass_math}")
percent_pass_reading = student_data.loc[student_data["reading_score"]>=70].shape[0]/tot_students
print(f"% Passing Reading: {percent_pass_reading}")

#convert the values to a pandas data frame
district_summary_df = pd.DataFrame({"Total Schools":[tot_schools], "Total Students":["{:,}".format(tot_students)],
                                   "Total Budget": ["${:,.2f}".format(tot_budget)], 
                                   "Average Math Score":[round(math_average, 2)],
                                   "Average Reading Score":[round(reading_average, 2)], 
                                   "%Passing Math":["{:.2f}%".format(percent_pass_math*100)],
                                   "%Passing Reading":["{:.2f}%".format(percent_pass_reading*100)], 
                                   "%Overall Passing Rate":[round(overall_average, 2)]})
district_summary_df


Schools: 15
Students:39170
Budget: 24649428
Average math: 78.98537145774827
Average reading: 81.87784018381414
Overall Average: 80.43160582078121
% Passing Math: 0.749808526933878
% Passing Reading: 0.8580546336482001


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,%Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,80.43


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [5]:
school_data_complete.head(2)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635


In [6]:
#use built-in pandas functions to aggregate the column data

school_aggs_df = pd.DataFrame(school_data_complete.groupby(["school_name", "type"]).aggregate({"student_name":"count", "budget":"first",
                                "math_score":"mean", "reading_score":"mean"})).reset_index()
school_math_pass = pd.DataFrame(school_data_complete.loc[school_data_complete["math_score"]>=70].groupby(["school_name", "type"]).aggregate({"math_score":"count"})).reset_index()
school_reading_pass = pd.DataFrame(school_data_complete.loc[school_data_complete["reading_score"]>=70].groupby(["school_name", "type"]).aggregate({"reading_score":"count"})).reset_index()
school_agg_summary = pd.merge(school_aggs_df, school_math_pass, how="left", on=["school_name", "type"])
school_agg_summary = pd.merge(school_agg_summary, school_reading_pass, how="left", on=["school_name", "type"])
school_agg_summary.head(5)

,school_name,type,student_name,budget,math_score_x,reading_score_x,math_score_y,reading_score_y
0,Bailey High School,District,4976,3124928,77.048432,81.033963,3318,4077
1,Cabrera High School,Charter,1858,1081356,83.061895,83.975780,1749,1803
2,Figueroa High School,District,2949,1884411,76.711767,81.158020,1946,2381
3,Ford High School,District,2739,1763916,77.102592,80.746258,1871,2172
4,Griffin High School,Charter,1468,917500,83.351499,83.816757,1371,1426


In [7]:
#give appropriate column names and data types to the summary data
school_agg_summary.columns= ["school_name", "type", "Total Students", "Total School Budget", "Average Math Score", "Average Reading Score",
                       "Students Passing Math", "Students Passing Reading"]
school_agg_summary[["Total Students", "Students Passing Math", "Students Passing Reading"]].astype(float, inplace=True)
school_agg_summary.head(5)


,school_name,type,Total Students,Total School Budget,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading
0,Bailey High School,District,4976,3124928,77.048432,81.033963,3318,4077
1,Cabrera High School,Charter,1858,1081356,83.061895,83.975780,1749,1803
2,Figueroa High School,District,2949,1884411,76.711767,81.158020,1946,2381
3,Ford High School,District,2739,1763916,77.102592,80.746258,1871,2172
4,Griffin High School,Charter,1468,917500,83.351499,83.816757,1371,1426


In [8]:
#calculate and append new columns

school_agg_summary["Per Student Budget"] = school_agg_summary["Total School Budget"]/school_agg_summary["Total Students"]
school_agg_summary["% Passing Math"] = school_agg_summary["Students Passing Math"]/school_agg_summary["Total Students"]
school_agg_summary["% Passing Reading"] = school_agg_summary["Students Passing Reading"]/school_agg_summary["Total Students"]
school_agg_summary["%Overall Passing Rate"] = (school_agg_summary["Students Passing Math"] + school_agg_summary["Students Passing Reading"])/(school_agg_summary["Total Students"]*2)
                                                                                     
school_agg_summary.head(5)

,school_name,type,Total Students,Total School Budget,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Per Student Budget,% Passing Math,% Passing Reading,%Overall Passing Rate
0,Bailey High School,District,4976,3124928,77.048432,81.033963,3318,4077,628.0,0.666801,0.819333,0.743067
1,Cabrera High School,Charter,1858,1081356,83.061895,83.975780,1749,1803,582.0,0.941335,0.970398,0.955867
2,Figueroa High School,District,2949,1884411,76.711767,81.158020,1946,2381,639.0,0.659885,0.807392,0.733639
3,Ford High School,District,2739,1763916,77.102592,80.746258,1871,2172,644.0,0.683096,0.792990,0.738043
4,Griffin High School,Charter,1468,917500,83.351499,83.816757,1371,1426,625.0,0.933924,0.971390,0.952657


In [9]:
school_agg_summary.columns

Index(['school_name', 'type', 'Total Students', 'Total School Budget',
       'Average Math Score', 'Average Reading Score', 'Students Passing Math',
       'Students Passing Reading', 'Per Student Budget', '% Passing Math',
       '% Passing Reading', '%Overall Passing Rate'],
      dtype='object')

In [10]:
#re-arrange columns
school_summary_df = school_agg_summary[['school_name', 'type', 'Total Students', 'Total School Budget',
       'Per Student Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math',
       '% Passing Reading', '%Overall Passing Rate']]



In [11]:
#using this section to demonstrate data formatting

school_summary_df["Total Students"] = school_summary_df["Total Students"].astype(int).map("{:,}".format)
school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].astype(float).map("${:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].astype(float).map("${:,.2f}".format)
school_summary_df.head(5)

C:\Users\deana\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\deana\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\deana\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is 

,school_name,type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,%Overall Passing Rate
0,Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,0.666801,0.819333,0.743067
1,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,0.941335,0.970398,0.955867
2,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,0.659885,0.807392,0.733639
3,Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,0.683096,0.792990,0.738043
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,0.933924,0.971390,0.952657


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [12]:
#sort the data and store a copy of the worst performin schools
worst_schools = school_summary_df.sort_values("%Overall Passing Rate", ascending=True).head(5)
worst_schools

,school_name,type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,%Overall Passing Rate
10,Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,0.663666,0.802201,0.732933
2,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,0.659885,0.807392,0.733639
7,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,0.656839,0.813164,0.735002
8,Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,0.660576,0.812224,0.736400
3,Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,0.683096,0.792990,0.738043


In [13]:
best_schools = school_summary_df.sort_values("%Overall Passing Rate", ascending=False).head(5)
best_schools

,school_name,type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,%Overall Passing Rate
1,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,0.941335,0.970398,0.955867
12,Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,0.932722,0.973089,0.952905
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,0.945946,0.959459,0.952703
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,0.933924,0.971390,0.952657
13,Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,0.938677,0.965396,0.952037


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [14]:
#pivot/'unstack' the data to achieve the above results
math_scores_by_grade_df = pd.DataFrame(school_data_complete.groupby(["school_name","grade"]).aggregate({"math_score":"mean"})).unstack(level=-1).reset_index()
math_scores_by_grade_df

school_name math_score                                 
grade                              10th       11th       12th        9th
0         Bailey High School  76.996772  77.515588  76.492218  77.083676
1        Cabrera High School  83.154506  82.765560  83.277487  83.094697
2       Figueroa High School  76.539974  76.884344  77.151369  76.403037
3           Ford High School  77.672316  76.918058  76.179963  77.361345
4        Griffin High School  84.229064  83.842105  83.356164  82.044010
5      Hernandez High School  77.337408  77.136029  77.186567  77.438495
6         Holden High School  83.429825  85.000000  82.855422  83.787402
7          Huang High School  75.908735  76.446602  77.225641  77.027251
8        Johnson High School  76.691117  77.491653  76.863248  77.187857
9           Pena High School  83.372000  84.328125  84.121547  83.625455
10     Rodriguez High School  76.612500  76.395626  77.690748  76.859966
11       Shelton High School  82.917411  83.383495  83.778976  83.420755
12        Thomas High School  83.087886  83.498795  83.497041  83.590022
13        Wilson High School  83.724422  83.195326  83.035794  83.085578
14        Wright High School  84.010288  83.836782  83.644986  83.264706

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [15]:
reading_scores_by_grade_df = pd.DataFrame(school_data_complete.groupby(["school_name","grade"]).aggregate({"reading_score":"mean"})).unstack(level=-1).reset_index()
reading_scores_by_grade_df

school_name reading_score                                 
grade                                 10th       11th       12th        9th
0         Bailey High School     80.907183  80.945643  80.912451  81.303155
1        Cabrera High School     84.253219  83.788382  84.287958  83.676136
2       Figueroa High School     81.408912  80.640339  81.384863  81.198598
3           Ford High School     81.262712  80.403642  80.662338  80.632653
4        Griffin High School     83.706897  84.288089  84.013699  83.369193
5      Hernandez High School     80.660147  81.396140  80.857143  80.866860
6         Holden High School     83.324561  83.815534  84.698795  83.677165
7          Huang High School     81.512386  81.417476  80.305983  81.290284
8        Johnson High School     80.773431  80.616027  81.227564  81.260714
9           Pena High School     83.612000  84.335938  84.591160  83.807273
10     Rodriguez High School     80.629808  80.864811  80.376426  80.993127
11       Shelton High School     83.441964  84.373786  82.781671  84.122642
12        Thomas High School     84.254157  83.585542  83.831361  83.728850
13        Wilson High School     84.021452  83.764608  84.317673  83.939778
14        Wright High School     83.812757  84.156322  84.073171  83.833333

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [16]:
# Sample bins. Feel free to create your own bins.
#spending_bins = [0, 585, 615, 645, 675]
#group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

#attempting to generate dynamic bin ranges as a personal challenge
converted_budget = pd.DataFrame(school_summary_df["Total School Budget"].str.replace("$", "").str.replace(",", "").astype(float))
converted_budget.columns
min_budget = converted_budget["Total School Budget"].min()
max_budget = converted_budget["Total School Budget"].max()
max_budget += max_budget*.05 
my_bins = list(range(0, int(max_budget) , int(max_budget/5)))
del my_bins[1]

#include backslash to include the $ sign
bin_names = ["\$" + str(curr_bin/100)[:len(str(curr_bin))-3] + "K to \$" + str(next_bin/1000)[:len(str(next_bin))-3] + "K" 
             for curr_bin, next_bin in zip(my_bins, my_bins[1:])]
#bin_names.append("Over $" + str(my_bins[-1:])[1:5] + "K")
my_bins                                                                                        
bin_names

['\\$0K to \\$1312K',
 '\\$1312K to \\$1968K',
 '\\$1968K to \\$2624K',
 '\\$2624K to \\$3281K']

In [17]:
#average by spending requires either going back to the complete recordset, or calculating the weighted average
#a simple mean would be skewed according to the number of students in any given bucket, as in if 100 students scored 70
#and 1 student scored 90, the simple mean on the summary would be 85, but the weighted average would be closer to 71
#I am opting for the complete data
scores_by_spending_df = school_data_complete
scores_by_spending_df["Spending Range"] = pd.cut(school_data_complete["budget"], my_bins, labels=bin_names)
scores_by_spending_df.head(5)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,Spending Range
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,\$1312K to \$1968K
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,\$1312K to \$1968K
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,\$1312K to \$1968K
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,\$1312K to \$1968K
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,\$1312K to \$1968K


In [18]:
#the total budget must also be summarized into the new buckets if it is to be determined
schools_by_spending_df = school_data
schools_by_spending_df["Spending Range"] = pd.cut(school_data["budget"], my_bins, labels=bin_names)
budgets_by_spending_df = pd.DataFrame(schools_by_spending_df.groupby(["Spending Range"]).agg({"budget":"sum"})).reset_index()
budgets_by_spending_df

,Spending Range,budget
0,\$0K to \$1312K,5981931
1,\$1312K to \$1968K,6878536
2,\$1968K to \$2624K,2547363
3,\$2624K to \$3281K,9241598


In [19]:
#perform calculations and consolidate result into a single dataframe
by_spending_summary_df = pd.DataFrame(scores_by_spending_df.groupby(["Spending Range"]).aggregate({"student_name":"count",
                                                        "math_score":"mean", "reading_score":"mean"})).reset_index()
math_pass_by_spending = pd.DataFrame(scores_by_spending_df.loc[scores_by_spending_df["math_score"]>=70].groupby(["Spending Range"]).aggregate({"math_score":"count"})).reset_index()
reading_pass_by_spending = pd.DataFrame(scores_by_spending_df.loc[scores_by_spending_df["reading_score"]>=70].groupby(["Spending Range"]).aggregate({"reading_score":"count"})).reset_index()
by_spending_summary_df = pd.merge(by_spending_summary_df, math_pass_by_spending, how="left", on="Spending Range")
by_spending_summary_df = pd.merge(by_spending_summary_df, reading_pass_by_spending, how="left", on="Spending Range")
by_spending_summary_df = pd.merge(by_spending_summary_df, budgets_by_spending_df, how="left", on="Spending Range")


by_spending_summary_df.columns= ["Spending Range", "Total Students",  "Average Math Score", "Average Reading Score",
                       "Students Passing Math", "Students Passing Reading", "Total Budget"]
by_spending_summary_df[["Total Students", "Students Passing Math", "Students Passing Reading"]].astype(float, inplace=True)

by_spending_summary_df

,Spending Range,Total Students,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Total Budget
0,\$0K to \$1312K,9911,83.436586,83.882857,9283,9581,5981931
1,\$1312K to \$1968K,10888,78.164034,81.654758,7876,9129,6878536
2,\$1968K to \$2624K,3999,76.842711,80.744686,2654,3208,2547363
3,\$2624K to \$3281K,14372,77.134219,80.979474,9557,11692,9241598


In [20]:
#Perform calculations and append to the aggregated records
by_spending_summary_df["Per Student Budget"] = round(by_spending_summary_df["Total Budget"]/by_spending_summary_df["Total Students"], 2)
by_spending_summary_df["% Passing Math"] = by_spending_summary_df["Students Passing Math"]/by_spending_summary_df["Total Students"]
by_spending_summary_df["% Passing Reading"] = by_spending_summary_df["Students Passing Reading"]/by_spending_summary_df["Total Students"]
by_spending_summary_df["%Overall Passing Rate"] = (by_spending_summary_df["Students Passing Math"] + by_spending_summary_df["Students Passing Reading"])/(by_spending_summary_df["Total Students"]*2)
                                                                                     
by_spending_summary_df

,Spending Range,Total Students,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Total Budget,Per Student Budget,% Passing Math,% Passing Reading,%Overall Passing Rate
0,\$0K to \$1312K,9911,83.436586,83.882857,9283,9581,5981931,603.56,0.936636,0.966704,0.951670
1,\$1312K to \$1968K,10888,78.164034,81.654758,7876,9129,6878536,631.75,0.723365,0.838446,0.780906
2,\$1968K to \$2624K,3999,76.842711,80.744686,2654,3208,2547363,637.00,0.663666,0.802201,0.732933
3,\$2624K to \$3281K,14372,77.134219,80.979474,9557,11692,9241598,643.03,0.664974,0.813526,0.739250


## Scores by School Size

* Perform the same operations as above, based on school size.

In [21]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1500, 3000, 5500]
group_names = ["Small (<1500)", "Medium (1500-3000)", "Large (3000-5500)"]

In [22]:
#assign the school data to buckets 
scores_by_size_df = school_data_complete
scores_by_size_df["Size Range"] = pd.cut(school_data_complete["size"], size_bins, labels=group_names)
scores_by_size_df.head(5)
scores_by_size_df 

#the total budget must also be summarized into the new buckets if it is to be determined
budgets_by_size_df = school_data
budgets_by_size_df["Size Range"] = pd.cut(school_data["size"], size_bins, labels=group_names)
budgets_by_size_df = pd.DataFrame(budgets_by_size_df.groupby(["Size Range"]).agg({"budget":"sum"})).reset_index()
budgets_by_size_df

,Size Range,budget
0,Small (<1500),1751445
1,Medium (1500-3000),11109022
2,Large (3000-5500),11788961


In [23]:
#perform size specific calculations

by_size_summary_df = pd.DataFrame(scores_by_size_df.groupby(["Size Range"]).aggregate({"student_name":"count",
                                                        "math_score":"mean", "reading_score":"mean"})).reset_index()
math_pass_by_size = pd.DataFrame(scores_by_size_df.loc[scores_by_size_df["math_score"]>=70].groupby(["Size Range"]).aggregate({"math_score":"count"})).reset_index()
reading_pass_by_size= pd.DataFrame(scores_by_size_df.loc[scores_by_size_df["reading_score"]>=70].groupby(["Size Range"]).aggregate({"reading_score":"count"})).reset_index()
by_size_summary_df = pd.merge(by_size_summary_df, math_pass_by_size, how="left", on="Size Range")
by_size_summary_df = pd.merge(by_size_summary_df, reading_pass_by_size, how="left", on="Size Range")
by_size_summary_df = pd.merge(by_size_summary_df, budgets_by_size_df, how="left", on="Size Range")


by_size_summary_df.columns= ["Size Range", "Total Students",  "Average Math Score", "Average Reading Score",
                       "Students Passing Math", "Students Passing Reading", "Total Budget"]
by_size_summary_df[["Total Students", "Students Passing Math", "Students Passing Reading"]].astype(float, inplace=True)

by_size_summary_df

,Size Range,Total Students,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Total Budget
0,Small (<1500),2857,83.583479,83.893245,2676,2760,1751445
1,Medium (1500-3000),17942,80.213577,82.529094,14483,15950,11109022
2,Large (3000-5500),18371,77.070764,80.928365,12211,14900,11788961


In [24]:
#perform and append calculations
by_size_summary_df["Per Student Budget"] = round(by_size_summary_df["Total Budget"]/by_size_summary_df["Total Students"], 2)
by_size_summary_df["% Passing Math"] = by_size_summary_df["Students Passing Math"]/by_size_summary_df["Total Students"]
by_size_summary_df["% Passing Reading"] = by_size_summary_df["Students Passing Reading"]/by_size_summary_df["Total Students"]
by_size_summary_df["%Overall Passing Rate"] = (by_size_summary_df["Students Passing Math"] + by_size_summary_df["Students Passing Reading"])/(by_size_summary_df["Total Students"]*2)
                                                                                     
by_size_summary_df

,Size Range,Total Students,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Total Budget,Per Student Budget,% Passing Math,% Passing Reading,%Overall Passing Rate
0,Small (<1500),2857,83.583479,83.893245,2676,2760,1751445,613.04,0.936647,0.966048,0.951348
1,Medium (1500-3000),17942,80.213577,82.529094,14483,15950,11109022,619.16,0.807212,0.888976,0.848094
2,Large (3000-5500),18371,77.070764,80.928365,12211,14900,11788961,641.72,0.664689,0.811061,0.737875


## Scores by School Type

* Perform the same operations as above, based on school type.

In [25]:
# No need for bins as type is already and attribute
scores_by_type_df = school_data_complete
scores_by_type_df.head(5)

#the total budget must also be summarized into the new buckets if it is to be determined
budgets_by_type_df = school_data
budgets_by_type_df = pd.DataFrame(budgets_by_type_df.groupby(["type"]).agg({"budget":"sum"})).reset_index()
budgets_by_type_df

,type,budget
0,Charter,7301505
1,District,17347923


In [110]:
by_type_summary_df = pd.DataFrame(scores_by_type_df.groupby(["type"]).aggregate({"student_name":"count",
                                                        "math_score":"mean", "reading_score":"mean"})).reset_index()
math_pass_by_type = pd.DataFrame(scores_by_type_df.loc[scores_by_type_df["math_score"]>=70].groupby(["type"]).aggregate({"math_score":"count"})).reset_index()
reading_pass_by_type= pd.DataFrame(scores_by_type_df.loc[scores_by_type_df["reading_score"]>=70].groupby(["type"]).aggregate({"reading_score":"count"})).reset_index()
by_type_summary_df = pd.merge(by_type_summary_df, math_pass_by_type, how="left", on="type")
by_type_summary_df = pd.merge(by_type_summary_df, reading_pass_by_type, how="left", on="type")
by_type_summary_df = pd.merge(by_type_summary_df, budgets_by_type_df, how="left", on="type")


by_type_summary_df.columns= ["type", "Total Students",  "Average Math Score", "Average Reading Score",
                       "Students Passing Math", "Students Passing Reading", "Total Budget"]
by_type_summary_df[["Total Students", "Students Passing Math", "Students Passing Reading"]].astype(float, inplace=True)

by_type_summary_df

,type,Total Students,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Total Budget
0,Charter,12194,83.406183,83.902821,11426,11785,7301505
1,District,26976,76.987026,80.962485,17944,21825,17347923


In [111]:
by_type_summary_df["Per Student Budget"] = round(by_type_summary_df["Total Budget"]/by_type_summary_df["Total Students"], 2)
by_type_summary_df["% Passing Math"] = by_type_summary_df["Students Passing Math"]/by_type_summary_df["Total Students"]
by_type_summary_df["% Passing Reading"] = by_type_summary_df["Students Passing Reading"]/by_type_summary_df["Total Students"]
by_type_summary_df["%Overall Passing Rate"] = (by_type_summary_df["Students Passing Math"] + 
                                                  by_type_summary_df["Students Passing Reading"]) / (by_type_summary_df["Total Students"]*2)
                                                                                     
by_type_summary_df

,type,Total Students,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Total Budget,Per Student Budget,% Passing Math,% Passing Reading,%Overall Passing Rate
0,Charter,12194,83.406183,83.902821,11426,11785,7301505,598.78,0.937018,0.966459,0.951739
1,District,26976,76.987026,80.962485,17944,21825,17347923,643.09,0.665184,0.809052,0.737118


In [112]:
#format the output
#for the sake of expedience, I am formatting the final output as a demonstration of the concepts
by_type_summary_df[["% Passing Math", "% Passing Reading", "%Overall Passing Rate"]] = by_type_summary_df.iloc[:,8:].applymap("{:,.2%}".format)
by_type_summary_df[["Total Students",  "Students Passing Math", "Students Passing Reading"]] = \
                    by_type_summary_df.loc[:,["Total Students",  "Students Passing Math", "Students Passing Reading"]].applymap("{:,}".format)
by_type_summary_df[["Average Math Score", "Average Reading Score"]] = by_type_summary_df.iloc[:,2:4].applymap("{:,.2f}".format)
by_type_summary_df[["Total Budget",  "Per Student Budget"]] = \
                    by_type_summary_df.loc[:,["Total Budget",  "Per Student Budget"]].applymap("${:,.2f}".format)

by_type_summary_df

,type,Total Students,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Total Budget,Per Student Budget,% Passing Math,% Passing Reading,%Overall Passing Rate
0,Charter,"12,194",83.41,83.90,"11,426","11,785","$7,301,505.00",$598.78,93.70%,96.65%,95.17%
1,District,"26,976",76.99,80.96,"17,944","21,825","$17,347,923.00",$643.09,66.52%,80.91%,73.71%


In [ ]:
"""Observations:
1. Performance does not correlate to higher budgets
2. Charter schools are vastly out-performing district schools
3. Smaller schools (by student population) does correlate to better performance
4. All the top performing schools are charter schools"""